### Readme:

This notebook documents what I learnt from https://course.spacy.io/en/

Special thanks to the content creators and the presenter Ines

If you want to learn more about spaCy, please visit https://spacy.io/ or https://course.spacy.io/en/

Thank you!

### Chapter 2: Large-scale data analysis with spaCy